# NBA Data-Insights
## The remarkable story of the rise of the 3-point shot
In the last 30 years of the NBA, the 3-point shot became increasingly popular. At the forefront of this rise stands Steph Curry, the baby-faced assasin from the Golden States Warriors. Few have made the 3-point focused playing style look so appealing as Curry. However, the 3-point shot has been on the rise long before Curry's first appearance (in 2009 with the Warriors). This notebook examines the story of the 3-point shot using four insights drawn from NBA data.

## Preparation
To prepare the notebook, the necessary module need to to be imported. 

In [ ]:
import sqlite3 as sql
from pathlib import Path
import nba.s01_data_wrangling as dw
import nba.s02_data_visualisation as dv
import warnings
warnings.filterwarnings("ignore")

A connection is subsequently established to the NBA database which hosts the data from which the insight are derived.

In [ ]:
DATA_PATH = Path.cwd() / 'data/nba.sqlite'
con = sql.connect(DATA_PATH)


## 1. Insight: Attempting 3-pointers is a winning strategy

We aim to examine whether attempting more 3-point shots makes winning more or less likely. To investigate this, we set up the following linear regression model:

$$
GameWon = B_0 + B_1 * fg3a
$$

- **GameWon**: A binary variable (0 or 1) indicating whether the team won (1) or lost (0).
- **fg3a**: The share of attempted shots that were 3-point attempts.

We run this regression for each year between 1990 and 2023 to obtain the coefficient (B1), showing how much 3-pointers contribute towards winning the game.

In [ ]:
# Load all games from the database
games = dw.select_games(con)
# Clean the data to get our regression data
data = dw.create_fg3a_data(games)
# Run the regressions for the years 1990 - 2023
regression_results = dw.run_regressions(data)

Having obtained the data, we create a time series plot indicating the value of the coefficient B1. The boundaries around the mean line show the 95%-standard-error band.

In [ ]:
args = {
    'data': regression_results,
    'y': 'fg3a_share_coefficient',
    'x': 'year',
    'se_min': 'se_min',
    'se_max': 'se_max',
    'regression': True,
    'title': 'Importance of 3-Pointers for Winning in the NBA',
    'xlabel': 'Year',
    'ylabel': 'Coefficient (B1) of Share-of-3-Pointer Attempts (fg3a)',
    'linecolor': '#FFC72C'
}
dv.time_series_plot(**args)

**Insight:** The graph clearly demonstrates how the 3-point shot became increasingly more important for winning over the last 30 years throughout the entire NBA.

# 2. Insight: NBA players slowly become better at shooting 3-pointers

With the 3-point shot becoming increasingly more relevant, one would expect that players also work on improving their 3-point shot. To test this, we examine how the success rate of 3-pointers (*three-point field goal efficacy*) changed since 1990.

In [ ]:
# Obtain the data containing the average 3-point shot success rate.
data = dw.get_avg_fg3_data(con)
print(data)

Having loaded in the data, we visualise the success rate using a time series plot. 

In [ ]:
# Visualise the data using a time series plot
args = {
    'data': data,
    'x': 'year',
    'y': 'avg_fg3_pct_home',
    'title': 'Success Rate of 3-Point-Shots in the NBA',
    'xlabel': 'Year',
    'regression': True,
    'ylabel': 'Average 3-Point NBA Percentage per Year',
    'linecolor': '#FFC72C'
}
dv.time_series_plot(**args)

**Insight:** Players clearly became better at shooting 3-pointers in the 90ies. However, since the early 2000s, players only slowly improve their 3-point shots.

# 3. Insight: Curry's Golden State Warriors dominated the Western Conference

The aforementioned Steph Curry, famous for his 3-point shot, plays since 2009 for the Golden State Warriors. The Golden State Warriors are interesting, as for 4 years in a row (between 2015-2019) they won the Western Conference. 

To track the dominance of the Warriors, we examine the win percentage of various Western Conference team using an MA(41) moving average process. 

$$ Win Percentage MA(41) = \frac{1}{41}\sum^{41}_{i}(GameWon_{i})$$

- **GameWon**: A binary variable (0 or 1) indicating whether the team won (1) or lost (0).

In a next step, we load the data for the Warriors and some other Western Conference teams.

<sup>Footnote: We are using an MA(41) process, as a regular NBA season consists of 82 games per team. Since we only track home games (i.e., half of all games), an MA(41) process provides the win percentage of a team across "one season"—or, in this case, the most recent 41 games.</sup>

In [ ]:
# Select the Western Conference Teams to analyse
teams = ["Warriors", "Lakers", "76ers", "Suns", "Nuggets", "Mavericks"]

# Add an MA41 column to the data
data = dw.get_ma41_data(con)
data = dw.clean_ma_data(data, teams)

print(data)

In [ ]:
# We visualise the data as a time series plot grouped by team
args = {
    "data": data,
    "x": "date",
    "y": "ma_41",
    "group_by": "team_name",
    "title": "Avg. Winning Rates of Western Conference Teams",
    "xlabel": "Year",
    "ylabel": "Moving Average (MA-41) of Win Percentage per Team",
    "main_team": "Warriors",
    "linecolor": "#FFC72C",
    "championship_years": [(2015, 2019), (2022, 2022)]
}

dv.multi_team_visualisation(**args)

**Insight:** Between 2015 and 2019, the Warriors were the dominant team in the Western Conference and won almost all of their games.

# 4. Insight: When the Warriors were dominant, they hit as many 3-pointers as none other

It was established that the Warriors had some very successful years between 2015 and 2019. An interesting question to ask, is whether in their dominant period, the Warriors were also better at shooting 3-pointers. 

To answer the question, the data is loaded first.

In [ ]:
# Load the data of average 3-point shot success rate per team
data = dw.get_3pt_success_rate(con, teams)
print(data)

In [ ]:
# Visualise the data using a time series plot grouped by team
args = {
    "data": data,
    "x": "year",
    "y": "avg_fg3_pct_home",
    "group_by": "team_nickname",
    "title": "Success Rate of 3-Point-Shots of Western Conference Teams",
    "xlabel": "Year",
    "ylabel": "Yearly Average 3-Point Percentage per Team",
    "main_team": "Warriors",
    "linecolor": "#FFC72C",
    "championship_years": [(2015, 2019), (2022, 2022)]
}

dv.multi_team_visualisation(**args)

**Insight:** The Warrios dominant years coincided with them sucessfully scoring more 3-pointers than any other team

In [ ]:
# We close the connection to the database
con.close()

## Conclusion
To conclude, the 3-point shot has seen a rising relevance in the NBA over the past 20 years. Its importance for winning games leads teams to continuously improve the 3-pointer skills of their players. A demonstration of the power of the 3-pointer is found in the case of the Golden State Warriors, which dominated the Western Conference and the NBA between 2015 - 2019. One significant factor of their dominance may be attributed to their team's exceptional ability to hit 3-point shots.